# Single Agent Organ Generator V1

This notebook demonstrates how to use the **Single Agent Organ Generator V1** workflow from the Agentic Organ Generation library. This workflow provides an interactive, LLM-driven process for generating organ vascular structures.

## What You'll Learn

1. How to set up and configure the workflow
2. Using the convenience function `run_single_agent_workflow()`
3. Programmatic control with `SingleAgentOrganGeneratorV1` class
4. State management (save/load workflow state)
5. Visualizing generated structures

## Workflow Overview

The Single Agent Organ Generator V1 follows these steps:

1. **INIT**: Ask for project name and output units
2. **REQUIREMENTS**: Ask for description of the desired structure
3. **GENERATING**: Use the library to generate structure and code
4. **VISUALIZING**: Output files and visualization of 3D object
5. **REVIEW**: Ask if the result is satisfactory
6. **CLARIFYING**: If not satisfied, ask for clarification (loops back to GENERATING)
7. **FINALIZING**: Output embedded structure, STL mesh, and generation code
8. **COMPLETE**: Close project

## Generated Artifacts

The workflow produces the following files:

- `design_spec.json` - Design specification used for generation
- `network.json` - Generated vascular network data
- `structure.stl` - STL mesh of the vascular structure
- `embedded_structure.stl` - Domain with vascular structure as negative space (printable scaffold)
- `generation_code.py` - Python code used to generate the structure
- `project_summary.json` - Summary of the project with all file paths

## 1. Setup and Installation

First, ensure you have the Agentic Organ Generation library installed.

In [ ]:
# Install the package if not already installed
# Uncomment the following line if needed:
# !pip install -e ..

# Verify imports work
import sys
import os

# Add parent directory to path if running from examples folder
sys.path.insert(0, os.path.abspath('..'))

# Test imports
from automation import (
    SingleAgentOrganGeneratorV1,
    WorkflowState,
    ProjectContext,
    run_single_agent_workflow,
    AgentRunner,
    LLMClient,
    create_agent,
)

print("All imports successful!")

## 2. Configuration

Configure your LLM provider and API key. The workflow supports:

- **OpenAI**: GPT-4, GPT-3.5 (set `OPENAI_API_KEY`)
- **Anthropic**: Claude 3 (set `ANTHROPIC_API_KEY`)
- **Local**: OpenAI-compatible local models

In [ ]:
# Configuration
PROVIDER = "openai"  # Options: "openai", "anthropic", "local"
MODEL = "gpt-4"  # Model name (e.g., "gpt-4", "claude-3-opus-20240229")
OUTPUT_DIR = "./projects"  # Base directory for project outputs

# Set your API key (or use environment variable)
# Option 1: Set environment variable
# os.environ["OPENAI_API_KEY"] = "your-api-key-here"
# os.environ["ANTHROPIC_API_KEY"] = "your-api-key-here"

# Option 2: Pass directly (shown in examples below)
API_KEY = os.environ.get("OPENAI_API_KEY") or os.environ.get("ANTHROPIC_API_KEY")

# Check if API key is available
if API_KEY:
    print(f"API key found for {PROVIDER}")
else:
    print("WARNING: No API key found. Set OPENAI_API_KEY or ANTHROPIC_API_KEY environment variable.")
    print("You can still explore the workflow structure without running LLM calls.")

## 3. Quick Start: Using `run_single_agent_workflow()`

The easiest way to use the workflow is with the convenience function. This runs an interactive session that prompts you for input.

**Note**: This function uses `input()` for user interaction, which works best in a local Jupyter environment.

In [ ]:
# Quick start example (interactive)
# Uncomment and run this cell to start an interactive workflow session

# if API_KEY:
#     context = run_single_agent_workflow(
#         provider=PROVIDER,
#         model=MODEL,
#         base_output_dir=OUTPUT_DIR,
#     )
#     print(f"\nProject completed: {context.project_name}")
#     print(f"Output directory: {context.output_dir}")
# else:
#     print("Skipping interactive workflow - no API key available")

## 4. Detailed Usage: `SingleAgentOrganGeneratorV1` Class

For more control, use the `SingleAgentOrganGeneratorV1` class directly. This allows you to:

- Access workflow state at any point
- Save and resume workflow sessions
- Programmatically control the workflow

In [ ]:
# Create the agent and workflow
if API_KEY:
    # Create agent with your configuration
    agent = create_agent(
        provider=PROVIDER,
        api_key=API_KEY,
        model=MODEL,
        output_dir=OUTPUT_DIR,
    )
    
    # Create workflow instance
    workflow = SingleAgentOrganGeneratorV1(
        agent=agent,
        base_output_dir=OUTPUT_DIR,
        verbose=True,
    )
    
    print(f"Workflow created: {workflow.WORKFLOW_NAME}")
    print(f"Version: {workflow.WORKFLOW_VERSION}")
    print(f"Current state: {workflow.get_state()}")
else:
    print("Skipping workflow creation - no API key available")
    workflow = None

### 4.1 Understanding Workflow States

The workflow progresses through several states. Let's explore them:

In [ ]:
# Explore workflow states
print("Available workflow states:")
for state in WorkflowState:
    print(f"  - {state.value}: {state.name}")

print("\nState descriptions:")
state_descriptions = {
    WorkflowState.INIT: "Ask for project name and output units",
    WorkflowState.REQUIREMENTS: "Ask for description of desired structure",
    WorkflowState.GENERATING: "Generate structure using LLM and library",
    WorkflowState.VISUALIZING: "Show generated files and visualization info",
    WorkflowState.REVIEW: "Ask if user is satisfied with result",
    WorkflowState.CLARIFYING: "Ask what's wrong and gather feedback",
    WorkflowState.FINALIZING: "Generate final outputs (embedded structure)",
    WorkflowState.COMPLETE: "Workflow finished",
}

for state, desc in state_descriptions.items():
    print(f"  {state.value}: {desc}")

### 4.2 Programmatic Control with `step()` Method

The `step()` method allows you to advance the workflow programmatically without interactive prompts. This is useful for:

- Automated testing
- Integration with other systems
- Notebook-based demonstrations

In [ ]:
# Demonstrate programmatic workflow control
# Note: step() handles state transitions but doesn't trigger LLM generation
# For full generation, use run() method

if workflow:
    # Check initial state
    print(f"Initial state: {workflow.get_state()}")
    
    # Step 1: Provide project name (INIT -> REQUIREMENTS)
    new_state, response = workflow.step("my_liver_project")
    print(f"After project name: {new_state.value}")
    print(f"Response: {response}")
    
    # Step 2: Provide description (REQUIREMENTS -> GENERATING)
    description = "Generate a liver vascular network with 500 arterial segments"
    new_state, response = workflow.step(description)
    print(f"After description: {new_state.value}")
    print(f"Response: {response}")
    
    # Check context
    context = workflow.get_context()
    print(f"\nProject context:")
    print(f"  Project name: {context.project_name}")
    print(f"  Description: {context.description}")
    print(f"  Output dir: {context.output_dir}")
else:
    print("Workflow not available - demonstrating state transitions conceptually")
    print("")
    print("Typical flow:")
    print("  1. INIT -> provide project name -> REQUIREMENTS")
    print("  2. REQUIREMENTS -> provide description -> GENERATING")
    print("  3. GENERATING -> (LLM generates) -> VISUALIZING")
    print("  4. VISUALIZING -> (auto) -> REVIEW")
    print("  5. REVIEW -> 'yes' -> FINALIZING")
    print("  5. REVIEW -> 'no' -> CLARIFYING -> GENERATING (loop)")
    print("  6. FINALIZING -> (LLM finalizes) -> COMPLETE")

### 4.3 Running the Full Workflow

To run the complete workflow with LLM generation, use the `run()` method. This is interactive and will prompt for input.

In [ ]:
# Run the full interactive workflow
# Uncomment to run (requires API key and interactive input)

# if API_KEY:
#     # Create a fresh workflow instance
#     agent = create_agent(
#         provider=PROVIDER,
#         api_key=API_KEY,
#         model=MODEL,
#     )
#     workflow = SingleAgentOrganGeneratorV1(
#         agent=agent,
#         base_output_dir=OUTPUT_DIR,
#     )
#     
#     # Run the workflow
#     final_context = workflow.run()
#     
#     # Access results
#     print(f"\nFinal artifacts:")
#     print(f"  STL mesh: {final_context.stl_path}")
#     print(f"  Embedded structure: {final_context.embedded_stl_path}")
#     print(f"  Generation code: {final_context.code_path}")

## 5. State Management: Save and Resume

The workflow supports saving and loading state, allowing you to pause and resume sessions.

In [ ]:
# Demonstrate state save/load
import tempfile
import json

if workflow:
    # Save current state
    state_file = os.path.join(tempfile.gettempdir(), "workflow_state.json")
    workflow.save_state(state_file)
    
    # View saved state
    with open(state_file, 'r') as f:
        saved_state = json.load(f)
    
    print("Saved state contents:")
    print(json.dumps(saved_state, indent=2))
    
    # Create new workflow and load state
    new_agent = create_agent(
        provider=PROVIDER,
        api_key=API_KEY,
        model=MODEL,
    )
    new_workflow = SingleAgentOrganGeneratorV1(
        agent=new_agent,
        base_output_dir=OUTPUT_DIR,
    )
    
    # Load saved state
    new_workflow.load_state(state_file)
    
    print(f"\nResumed workflow at state: {new_workflow.get_state()}")
    print(f"Project name: {new_workflow.get_context().project_name}")
else:
    print("Demonstrating state structure:")
    example_state = {
        "workflow_name": "Single Agent Organ Generator V1",
        "workflow_version": "1.0.0",
        "state": "requirements",
        "context": {
            "project_name": "my_project",
            "output_dir": "./projects/my_project",
            "description": "",
            "output_units": "mm",
            "iteration": 0,
            "feedback_history": []
        },
        "timestamp": 1234567890.0
    }
    print(json.dumps(example_state, indent=2))

## 6. Working with ProjectContext

The `ProjectContext` dataclass tracks all project information and generated artifacts.

In [ ]:
# Explore ProjectContext
import json

print("ProjectContext attributes:")
print("")

# Create example context
example_context = ProjectContext(
    project_name="liver_network_v1",
    output_dir="./projects/liver_network_v1",
    description="Generate a liver vascular network with dual arterial and venous trees",
    output_units="mm",
    spec_json="./projects/liver_network_v1/design_spec.json",
    network_json="./projects/liver_network_v1/network.json",
    stl_path="./projects/liver_network_v1/structure.stl",
    embedded_stl_path="./projects/liver_network_v1/embedded_structure.stl",
    code_path="./projects/liver_network_v1/generation_code.py",
    iteration=2,
    feedback_history=["Make the vessels thicker", "Add more branching"]
)

print(f"Project: {example_context.project_name}")
print(f"Output directory: {example_context.output_dir}")
print(f"Description: {example_context.description}")
print(f"Output units: {example_context.output_units}")
print(f"Iterations: {example_context.iteration}")
print(f"Feedback history: {example_context.feedback_history}")
print("")
print("Generated artifacts:")
print(f"  Design spec: {example_context.spec_json}")
print(f"  Network: {example_context.network_json}")
print(f"  STL mesh: {example_context.stl_path}")
print(f"  Embedded structure: {example_context.embedded_stl_path}")
print(f"  Generation code: {example_context.code_path}")

# Convert to dict
print("\nAs dictionary:")
print(json.dumps(example_context.to_dict(), indent=2))

## 7. Visualizing Generated Structures

After generation, you can visualize the STL meshes using `trimesh`.

In [ ]:
# Visualization utilities
import trimesh

def visualize_stl(stl_path, title="Mesh Visualization"):
    """
    Load and display an STL mesh.
    
    Parameters
    ----------
    stl_path : str
        Path to STL file
    title : str
        Title for the visualization
    """
    if not os.path.exists(stl_path):
        print(f"File not found: {stl_path}")
        return None
    
    # Load mesh
    mesh = trimesh.load(stl_path)
    
    # Print mesh info
    print(f"\n{title}")
    print(f"  File: {stl_path}")
    print(f"  Vertices: {len(mesh.vertices)}")
    print(f"  Faces: {len(mesh.faces)}")
    print(f"  Bounds: {mesh.bounds}")
    print(f"  Volume: {mesh.volume:.2f}")
    print(f"  Is watertight: {mesh.is_watertight}")
    
    return mesh

def show_mesh_interactive(mesh):
    """
    Show mesh in interactive viewer.
    
    Note: This requires a display environment.
    In Jupyter, you may need to use mesh.show() or create a Scene.
    """
    if mesh is None:
        return
    
    # Create scene for visualization
    scene = trimesh.Scene(mesh)
    
    # Try to show (may not work in all environments)
    try:
        scene.show()
    except Exception as e:
        print(f"Interactive visualization not available: {e}")
        print("To visualize, open the STL file in a 3D viewer like MeshLab or Blender.")

print("Visualization functions defined.")
print("")
print("Usage:")
print("  mesh = visualize_stl('path/to/structure.stl')")
print("  show_mesh_interactive(mesh)")

In [ ]:
# Example: Visualize generated structure (if available)
# Uncomment and modify path to visualize your generated structure

# stl_path = "./projects/my_project/structure.stl"
# if os.path.exists(stl_path):
#     mesh = visualize_stl(stl_path, "Generated Vascular Structure")
#     show_mesh_interactive(mesh)
# else:
#     print(f"No structure found at {stl_path}")
#     print("Run the workflow first to generate a structure.")

## 8. Using Different LLM Providers

The workflow supports multiple LLM providers. Here's how to switch between them:

In [ ]:
# Examples of different provider configurations

# OpenAI (GPT-4)
openai_config = {
    "provider": "openai",
    "model": "gpt-4",
    # API key from OPENAI_API_KEY env var or pass directly
}

# Anthropic (Claude 3)
anthropic_config = {
    "provider": "anthropic",
    "model": "claude-3-opus-20240229",
    # API key from ANTHROPIC_API_KEY env var or pass directly
}

# Local model (OpenAI-compatible API)
local_config = {
    "provider": "local",
    "model": "local-model",
    # Requires setting api_base in LLMConfig
}

print("Provider configurations:")
print("")
print("OpenAI:")
print(f"  {openai_config}")
print("")
print("Anthropic:")
print(f"  {anthropic_config}")
print("")
print("Local:")
print(f"  {local_config}")
print("")
print("Usage:")
print("  context = run_single_agent_workflow(**openai_config)")

## 9. Advanced: Custom Agent Configuration

For advanced use cases, you can customize the agent configuration.

In [ ]:
from automation.agent_runner import AgentConfig
from automation.llm_client import LLMConfig

# Custom LLM configuration
llm_config = LLMConfig(
    provider="openai",
    model="gpt-4",
    max_tokens=4096,
    temperature=0.7,
    system_prompt=None,  # Uses default system prompt
)

# Custom agent configuration
agent_config = AgentConfig(
    repo_path=".",
    output_dir="./output",
    max_iterations=10,
    auto_execute_code=False,
    verbose=True,
    save_conversation=True,
)

print("LLM Configuration:")
print(f"  Provider: {llm_config.provider}")
print(f"  Model: {llm_config.model}")
print(f"  Max tokens: {llm_config.max_tokens}")
print(f"  Temperature: {llm_config.temperature}")
print("")
print("Agent Configuration:")
print(f"  Max iterations: {agent_config.max_iterations}")
print(f"  Auto execute code: {agent_config.auto_execute_code}")
print(f"  Verbose: {agent_config.verbose}")
print(f"  Save conversation: {agent_config.save_conversation}")

In [ ]:
# Create agent with custom configuration
# Uncomment to use

# if API_KEY:
#     llm_config.api_key = API_KEY
#     
#     client = LLMClient(config=llm_config)
#     agent = AgentRunner(client=client, config=agent_config)
#     
#     workflow = SingleAgentOrganGeneratorV1(
#         agent=agent,
#         base_output_dir="./custom_projects",
#         verbose=True,
#     )
#     
#     print("Custom workflow created successfully!")

## 10. Appendix: Direct Generation Without LLM

If you want to generate structures without using an LLM, you can use the generation library directly.

In [ ]:
# Direct generation example (no LLM required)
from generation.api import run_experiment
from generation.specs import DesignSpec, TreeSpec

# Define a simple design specification
spec = DesignSpec(
    domain_type="ellipsoid",
    domain_params={"a": 0.05, "b": 0.04, "c": 0.03},  # 50mm x 40mm x 30mm (internal: meters)
    trees=[
        TreeSpec(
            name="arterial",
            inlet_position=(0, 0, 0.03),  # Top of ellipsoid
            inlet_radius=0.002,  # 2mm
            target_segments=100,
        )
    ],
    output_units="mm",
)

print("Design Specification:")
print(f"  Domain type: {spec.domain_type}")
print(f"  Domain params: {spec.domain_params}")
print(f"  Output units: {spec.output_units}")
print(f"  Trees: {len(spec.trees)}")
for tree in spec.trees:
    print(f"    - {tree.name}: {tree.target_segments} segments")

# Uncomment to run generation (may take a while)
# result = run_experiment(
#     spec=spec,
#     output_dir="./direct_generation_output",
#     manufacturing_config={
#         "min_channel_diameter": 0.5,
#         "min_wall_thickness": 0.3,
#         "plate_size": (200, 200, 200),
#     },
# )
# print(f"\nGeneration complete!")
# print(f"Domain mesh: {result.domain_mesh_path}")
# print(f"Surface mesh: {result.surface_mesh_path}")

## Summary

This notebook demonstrated:

1. **Quick Start**: Using `run_single_agent_workflow()` for interactive sessions
2. **Detailed Control**: Using `SingleAgentOrganGeneratorV1` class directly
3. **Programmatic Control**: Using `step()` method for state transitions
4. **State Management**: Saving and loading workflow state
5. **Visualization**: Loading and displaying generated STL meshes
6. **Provider Configuration**: Using different LLM providers
7. **Advanced Configuration**: Customizing agent and LLM settings
8. **Direct Generation**: Using the generation library without LLM

For more information, see:

- [Main README](../README.md)
- [Automation README](../automation/README.md)
- [Generation README](../generation/README.md)
- [Validity README](../validity/README.md)